In [1]:
code = r"""
#include <iostream>
#include <queue>
#include <omp.h>

using namespace std;

const int MAX = 1000;
int graph[MAX][MAX], visited[MAX];

void bfs(int start, int n) {
    queue<int> q;
    visited[start] = 1;
    q.push(start);

    while (!q.empty()) {
        int curr = q.front();
        q.pop();

        #pragma omp parallel for shared(graph, visited, q) default(none) firstprivate(curr, n)
        for (int i = 0; i < n; i++) {
            if (graph[curr][i]) {
                // Use atomic to avoid race condition on visited[i]
                if (!visited[i]) {
                    #pragma omp critical
                    {
                        if (!visited[i]) {
                            visited[i] = 1;
                            q.push(i);
                        }
                    }
                }
            }
        }
    }
}

int main() {
    int n, start;
          printf("#Prajwal Gadhave BACO21145");
        cout << endl;

    cout << "Enter number of vertices: ";
    cin >> n;

    cout << "Enter adjacency matrix:\n";
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            cin >> graph[i][j];
        }
    }

    cout << "Enter starting vertex (0-based index): ";
    cin >> start;

    bfs(start, n);

    cout << "BFS traversal: ";
    for (int i = 0; i < n; i++) {
        if (visited[i])
            cout << i << " ";
    }
    cout << endl;

    return 0;
}
"""

with open("bfs_parallel.cpp", "w") as f:
    f.write(code)


In [2]:
!g++ -fopenmp bfs_parallel.cpp -o bfs_parallel

In [3]:
!./bfs_parallel

#Prajwal Gadhave BACO21145
Enter number of vertices: 4
Enter adjacency matrix:
0 1 1 0 1 0 0 1 1 0 0 1 0 1 1 0
Enter starting vertex (0-based index): 0
BFS traversal: 0 1 2 3 


In [4]:
code = r"""
#include <iostream>
#include <stack>
#include <omp.h>
using namespace std;

const int MAX = 1000;
int graph[MAX][MAX], visited[MAX];

void dfs(int start, int n) {
    stack<int> s;
    s.push(start);

    while (!s.empty()) {
        int curr = s.top();
        s.pop();

        if (!visited[curr]) {
            visited[curr] = 1;

            // Parallelize this loop using OpenMP
            #pragma omp parallel for schedule(dynamic)
            for (int i = 0; i < n; i++) {
                if (graph[curr][i] && !visited[i]) {
                    #pragma omp critical
                    s.push(i);
                }
            }
        }
    }
}

int main() {
    int n, start;
          printf("#Prajwal Gadhave BACO21145");
        cout << endl;
    cout << "Enter number of vertices: ";
    cin >> n;

    cout << "Enter adjacency matrix:\n";
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            cin >> graph[i][j];
        }
    }

    cout << "Enter starting vertex: ";
    cin >> start;

    // Launching dfs in a parallel region
    #pragma omp parallel
    {
        #pragma omp single
        dfs(start, n);
    }

    cout << "DFS traversal: ";
    for (int i = 0; i < n; i++) {
        if (visited[i])
            cout << i << " ";
    }
    cout << endl;

    return 0;
}
"""

with open("dfs_parallel.cpp", "w") as f:
    f.write(code)


In [6]:
!g++ -fopenmp dfs_parallel.cpp -o dfs_parallel

In [8]:
!./dfs_parallel

#Prajwal Gadhave BACO21145
Enter number of vertices: 3
Enter adjacency matrix:
0 1 1 0 0 1 1 0 0
Enter starting vertex: 0
DFS traversal: 0 1 2 
